### In this notebook we connstruct a knn model using a test-train-validatin split which achieves:
### 62% accuracy classifying 'fam_or_subfam' (15 possible values) on unseen test data, and
### 87% accuracy classifying 'critter_name' (3 possible values).

In [9]:
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score
from sklearn.decomposition import TruncatedSVD

In [10]:
#Import the data file containing all features and classifications
df_all = pd.read_csv('MLNS_05292024.csv')

/var/folders/5z/7fkzpcz93gz6mqgh9vjgfymm0000gn/T/ipykernel_34325/2048960066.py:2: DtypeWarning: Columns (25,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv('MLNS_05292024.csv')


In [11]:
#Initial test-train split
df, df_test = train_test_split(df_all, shuffle=True, random_state=17, test_size=.2, stratify=df_all['fam_or_subfam'])

In [15]:
#Returns dataframe containing mfcc avg and var, and hs_mfcc avg and var, truncated to indicated depth, as well 
#as main_freq, range, max_mean, and peak_freq if True.
def truncate_mfcc(df, avg_depth=20, var_depth=10, hs_avg_depth=20, hs_var_depth=10, other_Features=True):
    df_temp=df['mfcc_'+str(0)+'_avg']
    df_temp=df_temp.rename('blah')
    for n in range(0,avg_depth):
        df_temp=pd.concat([df_temp, df['mfcc_'+str(n)+'_avg']], axis=1)
    for n in range(0,var_depth):
        df_temp=pd.concat([df_temp, df['mfcc_'+str(n)+'_var']], axis=1)
    for n in range(0,hs_avg_depth):
        df_temp=pd.concat([df_temp, df['hs_mfcc_'+str(n)+'_avg']], axis=1)
    for n in range(0,hs_var_depth):
        df_temp=pd.concat([df_temp, df['hs_mfcc_'+str(n)+'_var']], axis=1)
    if other_Features:
        df_temp=pd.concat([df_temp, df['main_freq']], axis=1)
        df_temp=pd.concat([df_temp, df['range']], axis=1)
        df_temp=pd.concat([df_temp, df['max_mean']], axis=1)
        df_temp=pd.concat([df_temp, df['peak_freq']], axis=1)
    return df_temp.drop(columns=['blah'])

In [16]:
#Computes the accuracy of knn at the given value of k, depths of mfcc features, other_features, and critter vs. fam_or_subfam
def knn_acc(df, k=5, avg_depth=40, var_depth=40, hs_avg_depth=40, hs_var_depth=40, other_Features=True, crit=False):
    X=truncate_mfcc(df=df, avg_depth=avg_depth,var_depth=var_depth,hs_avg_depth=hs_avg_depth,hs_var_depth=hs_var_depth,other_Features=other_Features)
    if crit:
        Y = df['critter_name']
    else:
        Y = df['fam_or_subfam']
    X_train, X_val, y_train, y_val = train_test_split(X.copy(), Y, shuffle=True, random_state=17, test_size=.2, stratify=Y)
    knn_pipe = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsClassifier(k))])
    knn_pipe.fit(X_train, y_train)
    pred_train = knn_pipe.predict(X_train)
    pred_val = knn_pipe.predict(X_val)
    score_train = accuracy_score(y_train,pred_train)
    score_val = accuracy_score(y_val,pred_val)
    return score_train, score_val

### The data below shows that using all of our features narrows the gap between training accuracy and validation accuracy.

In [18]:
#Runs knn over the indicated depth of all 4 mfcc features, including the 4 other features, and the values of k for knn.
for depth in range(1,40,5):
    for k in range(1,30,5):
        print(f"knn (Train, Validation) accuracy when mfcc depth = {depth} and k = {k}: {knn_acc(df,k,depth,depth,depth,depth,True,False)}") 

knn (Train, Validation) accuracy when mfcc depth = 1 and k = 1: (1.0, 0.5021186440677966)
knn (Train, Validation) accuracy when mfcc depth = 1 and k = 6: (0.6566887417218543, 0.565677966101695)
knn (Train, Validation) accuracy when mfcc depth = 1 and k = 11: (0.6164238410596027, 0.5625)
knn (Train, Validation) accuracy when mfcc depth = 1 and k = 16: (0.5957615894039735, 0.573093220338983)
knn (Train, Validation) accuracy when mfcc depth = 1 and k = 21: (0.584635761589404, 0.5677966101694916)
knn (Train, Validation) accuracy when mfcc depth = 1 and k = 26: (0.5732450331125828, 0.5667372881355932)
knn (Train, Validation) accuracy when mfcc depth = 6 and k = 1: (1.0, 0.6101694915254238)
knn (Train, Validation) accuracy when mfcc depth = 6 and k = 6: (0.7144370860927153, 0.628177966101695)
knn (Train, Validation) accuracy when mfcc depth = 6 and k = 11: (0.6749668874172186, 0.621822033898305)
knn (Train, Validation) accuracy when mfcc depth = 6 and k = 16: (0.6500662251655629, 0.619703389

### We now use all of our features and try to find value of k which maximizes the validation accuracy.

In [19]:
for k in range(1,50):
    print(f"knn (Train, Validation) accuracy when k = {k}: {knn_acc(df,k,40,40,40,40,True,False)}") 

knn Validation accuracy when k = 1: (1.0, 0.6292372881355932)
knn Validation accuracy when k = 2: (0.8013245033112583, 0.611228813559322)
knn Validation accuracy when k = 3: (0.7737748344370861, 0.6197033898305084)
knn Validation accuracy when k = 4: (0.7557615894039735, 0.621822033898305)
knn Validation accuracy when k = 5: (0.7385430463576159, 0.635593220338983)
knn Validation accuracy when k = 6: (0.7160264900662252, 0.6154661016949152)
knn Validation accuracy when k = 7: (0.7086092715231788, 0.6313559322033898)
knn Validation accuracy when k = 8: (0.697748344370861, 0.621822033898305)
knn Validation accuracy when k = 9: (0.6887417218543046, 0.625)
knn Validation accuracy when k = 10: (0.6786754966887417, 0.6324152542372882)
knn Validation accuracy when k = 11: (0.6778807947019867, 0.6292372881355932)
knn Validation accuracy when k = 12: (0.6744370860927152, 0.6260593220338984)
knn Validation accuracy when k = 13: (0.6688741721854304, 0.6292372881355932)
knn Validation accuracy when

### It appears that k=5 gives the best validation accuracy. We now test our final knn model using k=5 and all features.

In [24]:
#First train the model exactly using the same training data as before, and k=5.
knn_pipe = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsClassifier(5))])
X=truncate_mfcc(df=df, avg_depth=40,var_depth=40,hs_avg_depth=40,hs_var_depth=40,other_Features=True)
Y = df['fam_or_subfam']
X_train, X_val, y_train, y_val = train_test_split(X.copy(), Y, shuffle=True, random_state=17, test_size=.2, stratify=Y)
knn_pipe.fit(X_train, y_train)

#Test the model on unseen test data.
X_test=truncate_mfcc(df=df_test, avg_depth=40,var_depth=40,hs_avg_depth=40,hs_var_depth=40,other_Features=True)
y_test=df_test['fam_or_subfam']
pred = knn_pipe.predict(X_test)
accuracy_score(pred,y_test)

0.6203389830508474

### Our knn model with k=5 achieves 62% accuracy identifying 'fam_or_subfam' on unseen test data. Finally, we test our model on identifying critter_name: 'cricket', 'kaydid', or 'cicada'.

In [25]:
#Dictionary from 'fam_or_subfam' to the coarser classification 'critter_name'
fam_dict = {'Gryllinae':'cricket', 'Conocephalinae':'kaydid', 'Oecanthinae':'cricket',
            'Phaneropterinae': 'kaydid', 'Trigonidiinae':'cricket', 'Nemobiinae':'cricket', 'Hapithinae':'cricket', 
            'Mogoplistinae':'cricket', 'Tettigoniinae':'kaydid', 'Pseudophyllinae':'kaydid', 'Cicadidae':'cicada',
            'Gryllotalpidae':'cricket', 'Eneopterinae':'cricket', 'Phalangopsidae':'cricket', 'Listroscelidinae':'cricket'}

#Converts a pd Series with fam_or_subfam entries into critter names
def fam_to_crit(series):
    X=series.copy()
    for i in range(0, X.shape[0]):
        X.at[i]=fam_to_crit_string(X[i])
    return X

def fam_to_crit_string(fam_name):
    return fam_dict[fam_name]

In [26]:
accuracy_score(fam_to_crit(pd.Series(list(y_test))),fam_to_crit(pd.Series(pred)))

0.8694915254237288

### Our knn model with k=5 achieves 87% accuracy identifying (cricket, kaydid, or cicada) on unseen test data.